In [1]:
# Import Dependencies
import pandas as pd
import requests
import json

In [2]:
# Create base url
base_url = "https://crashviewer.nhtsa.dot.gov/CrashAPI/analytics/GetInjurySeverityCounts?"

# Store parameters
fromYear = 2015
toYear = 2015

# build partial query url
partial_url = f"{base_url}fromCaseYear={fromYear}&toCaseYear={toYear}"

In [3]:
# Set up empty list to store data
state_ids = []
total_fatals = []

# Run query 50 times, changing 'state=' to pull data from all 50 states
for n in range(1,51):
    query_url = f"{partial_url}&state={n}&format=json"
    
    try:
        print(f"Querying state {n}...")
        response = requests.get(query_url).json()
        state_ids.append(n)
        total_fatals.append(response["Results"][0][0]["TotalFatalCounts"])
        
    except:
        print(f"State {n} not found. Trying next state...")
        total_fatals.append(None)

Querying state 1...
Querying state 2...
Querying state 3...
State 3 not found. Trying next state...
Querying state 4...
Querying state 5...
Querying state 6...
Querying state 7...
State 7 not found. Trying next state...
Querying state 8...
Querying state 9...
Querying state 10...
Querying state 11...
Querying state 12...
Querying state 13...
Querying state 14...
State 14 not found. Trying next state...
Querying state 15...
Querying state 16...
Querying state 17...
Querying state 18...
Querying state 19...
Querying state 20...
Querying state 21...
Querying state 22...
Querying state 23...
Querying state 24...
Querying state 25...
Querying state 26...
Querying state 27...
Querying state 28...
Querying state 29...
Querying state 30...
Querying state 31...
Querying state 32...
Querying state 33...
Querying state 34...
Querying state 35...
Querying state 36...
Querying state 37...
Querying state 38...
Querying state 39...
Querying state 40...
Querying state 41...
Querying state 42...
Queryi

In [7]:
# Create dictionary from lists
accident_dict = {
    "state_id": state_ids,
    "total_fatalaties": total_fatals
}

accidentData = pd.DataFrame(accident_dict)

In [9]:
# Read in binge drinking data
binge_drinking = pd.read_csv("Resources/BingeDrinkingPrealence.csv")

# Preview dataset
binge_drinking.head()

,State,Percentage
0,AL,12.2
1,AK,20.0
2,AZ,15.0
3,AR,15.2
4,CA,16.7


In [10]:
# Read in state gender distribution data
gender_dist = pd.read_csv("Resources/StatePop_GenderDist.csv")

# Preview dataset
gender_dist.head()

,Location,Male,Female,Total,Footnotes
0,United States,152799800,159978200,312777900,1.0
1,Alabama,2276300,2453800,4730100,NaN
2,Alaska,363500,344000,707500,NaN
3,Arizona,3260100,3395100,6655100,NaN
4,Arkansas,1401000,1481300,2882300,NaN


In [11]:
# find gender ratio, drop US total from data
gender_dist=gender_dist.drop(0)

In [12]:
# extract only the data we need from gender_dist table
gender_ratio = (gender_dist['Male']/gender_dist['Female'])

In [13]:
# Merge the data into a single set
merged_data=pd.merge(accidentData, binge_drinking, right_index=True, left_index=True)
merged_data=merged_data.set_index('state_id')
merged_data.head()

,total_fatalaties,State,Percentage
state_id,,,
1,850.0,AL,12.2
2,65.0,AK,20.0
3,NaN,AZ,15.0
4,897.0,AR,15.2
5,550.0,CA,16.7


In [14]:
# Add in gender ratio data
merged_data['Male/Female']=gender_ratio

In [15]:
# final cleanup
merged_data=merged_data[['State', 'Male/Female', 'Percentage', 'total_fatalaties']]
merged_data=merged_data.rename(columns={'Percentage': 'Binge Drinking %', 'total_fatalaties': 'Total Vehicle Fatalaties'})
merged_data.head()

,State,Male/Female,Binge Drinking %,Total Vehicle Fatalaties
state_id,,,,
1,AL,0.927663,12.2,850.0
2,AK,1.056686,20.0,65.0
3,AZ,0.960237,15.0,NaN
4,AR,0.945791,15.2,897.0
5,CA,0.971978,16.7,550.0
